In [1]:
pip install XlsxWriter

In [30]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 89849765, 'calculationPrice': 'close', 'change': 1.18, 'changePercent': 0.00844, 'close': 0, 'closeSource': 'icfilafo', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 144.58, 'iexCloseTime': 1669827947076, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 146.74, 'iexOpenTime': 1644631779759, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1657760727204, 'latestPrice': 144.65, 'latestSource': 'Close', 'latestTime': 'October 1, 2021', 'latestUpdate': 1648260175143, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2432896399773, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime

In [6]:
#parsing API call
price = data['latestPrice']
pe_ratio=data['peRatio']
pe_ratio

27.99

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [8]:
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy


In [9]:
for symbol_string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe=final_dataframe.append(
        pd.Series(
            [
              symbol,
              data[symbol]['quote']['latestPrice'],
              data[symbol]['quote']['peRatio'],
              'N\A'
            ],
            index = my_columns
        ),
        ignore_index=True
    )

    

In [10]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,160.12,49.1,N\A
1,AAL,21.93,-2.26,N\A
2,AAP,215.00,23.06,N\A
3,AAPL,149.14,28.09,N\A
4,ABBV,112.21,29.4,N\A
...,...,...,...,...
500,YUM,125.41,29.41,N\A
501,ZBH,156.38,34.81,N\A
502,ZBRA,533.90,37.35,N\A
503,ZION,66.32,6.5,N\A


In [11]:
#removing glamour stocks
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,72.30,3.77,N\A
1,MPC,64.20,5.41,N\A
2,BIO,750.25,5.75,N\A
3,PRU,107.95,6.05,N\A
4,GM,55.16,6.4,N\A
5,ZION,66.32,6.5,N\A
6,AFL,53.53,6.58,N\A
7,COF,167.70,6.65,N\A
8,CINF,120.51,6.67,N\A
9,WHR,213.18,7.06,N\A


In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [13]:
portfolio_input()

Enter the value of your portfolio:1000000


In [14]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size

20000.0

In [15]:
for row in final_dataframe.index:
  final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,72.30,3.77,276
1,MPC,64.20,5.41,311
2,BIO,750.25,5.75,26
3,PRU,107.95,6.05,185
4,GM,55.16,6.4,362
5,ZION,66.32,6.5,301
6,AFL,53.53,6.58,373
7,COF,167.70,6.65,119
8,CINF,120.51,6.67,165
9,WHR,213.18,7.06,93


In [33]:
#adding more parameters
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data=requests.get(batch_api_call_url).json()

# Price to earning ration
pe_ratio = data[symbol]['quote']['peRatio']

#Price to book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price to sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#EV/EBITDA
ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']

#EV/GP
ev_to_gross_profit  = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']


In [41]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'    
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
  data=requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    try: 
      ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
    except TypeError:
      ev_to_ebitda = np.NaN
    try: 
      ev_to_gross_profit = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
    except TypeError:
      ev_to_gross_profit = np.NaN
    rv_dataframe = rv_dataframe.append(
    pd.Series([
        symbol,
        data[symbol]['quote']['latestPrice'],
        'N/A',
        data[symbol]['quote']['peRatio'],
        'N/A',
        data[symbol]['advanced-stats']['priceToBook'],
        'N/A',
        data[symbol]['advanced-stats']['priceToSales'],
        'N/A',
        ev_to_ebitda,
        'N/A',
        ev_to_gross_profit,
        'N/A',
        'N/A'    
    ],
    index = rv_columns),
        ignore_index=True
    )


In [42]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.63,N/A,49.3,N/A,9.88,N/A,7.95,N/A,31.782077,N/A,15.279758,N/A,N/A
1,AAL,22.04,N/A,-2.28,N/A,-1.91,N/A,0.7816,N/A,-4.744793,N/A,1.921669,N/A,N/A
2,AAP,216.00,N/A,23.38,N/A,4.17,N/A,1.2,N/A,11.318666,N/A,2.683972,N/A,N/A
3,AAPL,146.40,N/A,28.95,N/A,37.57,N/A,6.91,N/A,22.220943,N/A,17.260178,N/A,N/A
4,ABBV,110.32,N/A,30.6,N/A,15.43,N/A,3.62,N/A,10.795950,N/A,7.814483,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.31,N/A,28.77,N/A,-4.68,N/A,5.98,N/A,20.990933,N/A,10.025642,N/A,N/A
501,ZBH,152.29,N/A,35.71,N/A,2.57,N/A,4,N/A,20.271731,N/A,6.692895,N/A,N/A
502,ZBRA,514.88,N/A,36.46,N/A,10.65,N/A,5.46,N/A,27.719073,N/A,11.644001,N/A,N/A
503,ZION,66.79,N/A,6.6,N/A,1.39,N/A,2.99,N/A,5.084088,N/A,2.727192,N/A,N/A


In [44]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.90,N/A,61.39,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,295.27,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,286.23,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.85,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.19,N/A,14.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.89,N/A,33.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,38.64,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2771.10,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,108.15,N/A,35.11,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [45]:
rv_dataframe.columns


Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [51]:
for column in [ 'Price-to-Earnings Ratio', 'Price-to-Book Ratio',
       'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP' ]:
  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [53]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.63,N/A,49.30,N/A,9.88,N/A,7.9500,N/A,31.782077,N/A,15.279758,N/A,N/A
1,AAL,22.04,N/A,-2.28,N/A,-1.91,N/A,0.7816,N/A,-4.744793,N/A,1.921669,N/A,N/A
2,AAP,216.00,N/A,23.38,N/A,4.17,N/A,1.2000,N/A,11.318666,N/A,2.683972,N/A,N/A
3,AAPL,146.40,N/A,28.95,N/A,37.57,N/A,6.9100,N/A,22.220943,N/A,17.260178,N/A,N/A
4,ABBV,110.32,N/A,30.60,N/A,15.43,N/A,3.6200,N/A,10.795950,N/A,7.814483,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.31,N/A,28.77,N/A,-4.68,N/A,5.9800,N/A,20.990933,N/A,10.025642,N/A,N/A
501,ZBH,152.29,N/A,35.71,N/A,2.57,N/A,4.0000,N/A,20.271731,N/A,6.692895,N/A,N/A
502,ZBRA,514.88,N/A,36.46,N/A,10.65,N/A,5.4600,N/A,27.719073,N/A,11.644001,N/A,N/A
503,ZION,66.79,N/A,6.60,N/A,1.39,N/A,2.9900,N/A,5.084088,N/A,2.727192,N/A,N/A


In [52]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [59]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}

for metric in metrics.keys():
  for row in rv_dataframe.index:
    rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.63,N/A,49.30,83.9604,9.88,80.7921,7.9500,82.2772,31.782077,88.5149,15.279758,84.7525,N/A
1,AAL,22.04,N/A,-2.28,7.52475,-1.91,4.9505,0.7816,7.72277,-4.744793,2.77228,1.921669,8.51485,N/A
2,AAP,216.00,N/A,23.38,46.5347,4.17,54.7525,1.2000,15.3465,11.318666,33.0693,2.683972,12.6733,N/A
3,AAPL,146.40,N/A,28.95,58.6139,37.57,96.0396,6.9100,78.6139,22.220943,72.6733,17.260178,88.7129,N/A
4,ABBV,110.32,N/A,30.60,62.0792,15.43,89.1089,3.6200,52.2772,10.795950,30.297,7.814483,52.4752,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.31,N/A,28.77,58.0198,-4.68,4.55446,5.9800,71.3861,20.990933,70.8911,10.025642,66.9307,N/A
501,ZBH,152.29,N/A,35.71,71.0891,2.57,34.4554,4.0000,56.8317,20.271731,69.703,6.692895,41.7822,N/A
502,ZBRA,514.88,N/A,36.46,72.4752,10.65,81.9802,5.4600,67.9208,27.719073,84.7525,11.644001,74.4554,N/A
503,ZION,66.79,N/A,6.60,9.40594,1.39,13.4653,2.9900,43.9604,5.084088,5.34653,2.727192,12.8713,N/A


In [61]:
#RV Score
from statistics import mean

for row in rv_dataframe.index:
  value_percentiles = []
  for metric in metrics.keys():
    value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
  rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.63,N/A,49.30,83.9604,9.88,80.7921,7.9500,82.2772,31.782077,88.5149,15.279758,84.7525,84.0594
1,AAL,22.04,N/A,-2.28,7.52475,-1.91,4.9505,0.7816,7.72277,-4.744793,2.77228,1.921669,8.51485,6.29703
2,AAP,216.00,N/A,23.38,46.5347,4.17,54.7525,1.2000,15.3465,11.318666,33.0693,2.683972,12.6733,32.4752
3,AAPL,146.40,N/A,28.95,58.6139,37.57,96.0396,6.9100,78.6139,22.220943,72.6733,17.260178,88.7129,78.9307
4,ABBV,110.32,N/A,30.60,62.0792,15.43,89.1089,3.6200,52.2772,10.795950,30.297,7.814483,52.4752,57.2475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.31,N/A,28.77,58.0198,-4.68,4.55446,5.9800,71.3861,20.990933,70.8911,10.025642,66.9307,54.3564
501,ZBH,152.29,N/A,35.71,71.0891,2.57,34.4554,4.0000,56.8317,20.271731,69.703,6.692895,41.7822,54.7723
502,ZBRA,514.88,N/A,36.46,72.4752,10.65,81.9802,5.4600,67.9208,27.719073,84.7525,11.644001,74.4554,76.3168
503,ZION,66.79,N/A,6.60,9.40594,1.39,13.4653,2.9900,43.9604,5.084088,5.34653,2.727192,12.8713,17.0099


In [63]:
#selecting 50 best
rv_dataframe.sort_values('RV Score', ascending = True, inplace= True)
rv_dataframe = rv_dataframe[:50]


In [64]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
459,UNM,26.15,N/A,7.84,11.6832,0.4903,5.14851,0.4023,2.17822,3.511448,3.76238,0.410073,0.990099,4.75248
381,PRU,109.24,N/A,6.00,8.71287,0.6552,5.54455,0.6573,5.94059,4.618327,4.55446,0.638186,1.38614,5.22772
1,AAL,22.04,N/A,-2.28,7.52475,-1.9100,4.9505,0.7816,7.72277,-4.744793,2.77228,1.921669,8.51485,6.29703
300,MCK,206.36,N/A,-7.32,5.74257,-58.4900,1.38614,0.1281,0.792079,7.947789,17.8218,2.780191,13.4653,7.84158
274,L,56.40,N/A,9.69,14.2574,0.7924,6.73267,0.9628,10.099,5.425318,6.13861,0.998376,3.16832,8.07921
391,RE,259.02,N/A,7.55,10.6931,0.9871,7.32673,0.9313,9.50495,6.781692,10.6931,0.915673,2.77228,8.19802
272,KSS,50.59,N/A,10.60,16.4356,1.5000,16.9307,0.3987,1.9802,4.167115,3.9604,1.175270,3.9604,8.65347
55,BA,233.00,N/A,-14.89,4.75248,-8.0500,4.15842,2.2400,34.5545,-31.105610,0.792079,-216.941157,0.39604,8.93069
26,ALL,132.65,N/A,10.27,15.8416,1.4700,15.8416,0.7902,7.92079,2.423742,3.36634,0.770153,2.17822,9.0297
203,GM,54.20,N/A,6.16,8.91089,1.5100,17.6238,0.5622,4.75248,2.431336,3.56436,2.151354,10.297,9.0297


In [65]:
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.15,N/A,7.84,11.6832,0.4903,5.14851,0.4023,2.17822,3.511448,3.76238,0.410073,0.990099,4.75248
1,PRU,109.24,N/A,6.00,8.71287,0.6552,5.54455,0.6573,5.94059,4.618327,4.55446,0.638186,1.38614,5.22772
2,AAL,22.04,N/A,-2.28,7.52475,-1.9100,4.9505,0.7816,7.72277,-4.744793,2.77228,1.921669,8.51485,6.29703
3,MCK,206.36,N/A,-7.32,5.74257,-58.4900,1.38614,0.1281,0.792079,7.947789,17.8218,2.780191,13.4653,7.84158
4,L,56.40,N/A,9.69,14.2574,0.7924,6.73267,0.9628,10.099,5.425318,6.13861,0.998376,3.16832,8.07921
5,RE,259.02,N/A,7.55,10.6931,0.9871,7.32673,0.9313,9.50495,6.781692,10.6931,0.915673,2.77228,8.19802
6,KSS,50.59,N/A,10.60,16.4356,1.5000,16.9307,0.3987,1.9802,4.167115,3.9604,1.175270,3.9604,8.65347
7,BA,233.00,N/A,-14.89,4.75248,-8.0500,4.15842,2.2400,34.5545,-31.105610,0.792079,-216.941157,0.39604,8.93069
8,ALL,132.65,N/A,10.27,15.8416,1.4700,15.8416,0.7902,7.92079,2.423742,3.36634,0.770153,2.17822,9.0297
9,GM,54.20,N/A,6.16,8.91089,1.5100,17.6238,0.5622,4.75248,2.431336,3.56436,2.151354,10.297,9.0297


In [68]:
#calculating number of shares to buy
portfolio_input()

Enter the value of your portfolio:2500000


In [69]:
print(portfolio_size)

2500000


In [71]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
position_size

50000.0

In [75]:
for row in rv_dataframe.index:
  rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row,'Price'])

rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.15,1912,7.84,11.6832,0.4903,5.14851,0.4023,2.17822,3.511448,3.76238,0.410073,0.990099,4.75248
1,PRU,109.24,457,6.00,8.71287,0.6552,5.54455,0.6573,5.94059,4.618327,4.55446,0.638186,1.38614,5.22772
2,AAL,22.04,2268,-2.28,7.52475,-1.9100,4.9505,0.7816,7.72277,-4.744793,2.77228,1.921669,8.51485,6.29703
3,MCK,206.36,242,-7.32,5.74257,-58.4900,1.38614,0.1281,0.792079,7.947789,17.8218,2.780191,13.4653,7.84158
4,L,56.40,886,9.69,14.2574,0.7924,6.73267,0.9628,10.099,5.425318,6.13861,0.998376,3.16832,8.07921
5,RE,259.02,193,7.55,10.6931,0.9871,7.32673,0.9313,9.50495,6.781692,10.6931,0.915673,2.77228,8.19802
6,KSS,50.59,988,10.60,16.4356,1.5000,16.9307,0.3987,1.9802,4.167115,3.9604,1.175270,3.9604,8.65347
7,BA,233.00,214,-14.89,4.75248,-8.0500,4.15842,2.2400,34.5545,-31.105610,0.792079,-216.941157,0.39604,8.93069
8,ALL,132.65,376,10.27,15.8416,1.4700,15.8416,0.7902,7.92079,2.423742,3.36634,0.770153,2.17822,9.0297
9,GM,54.20,922,6.16,8.91089,1.5100,17.6238,0.5622,4.75248,2.431336,3.56436,2.151354,10.297,9.0297
